# 4. Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import os
from pickle import load
import pickle
import DiadFit as pf
pf.__version__

'0.0.59'

In [2]:
MasterFolder = r"P:\WORK-GENERAL\POSTDOC-UCB\BERKELEY-VIBE\Documents\Projects\Data\Hawaii_FI\Data\Raman\Leilani-2018"
DayFolder = os.path.join(MasterFolder, "Leilani_2018FI_Oct2522pictures for review")

meta_path=DayFolder +  '\Metadata'
spectra_path=DayFolder + '\Spectra'
filetype='headless_txt'

if not os.path.exists(MasterFolder + '\OUTPUT'):
    os.mkdir(MasterFolder +'\OUTPUT')


output_path=MasterFolder +'\OUTPUT'


## Load in specra, and combine

In [3]:
from os import path
if path.exists('Discarded_df.xlsx'):
    discard=pd.read_excel('Discarded_df.xlsx')
else:
    discard=None
if path.exists('Weak_Diads.xlsx'):
    grp1=pd.read_excel('Weak_Diads.xlsx')
else:
    grp1=None
if path.exists('Medium_Diads.xlsx'):
    grp2=pd.read_excel('Medium_Diads.xlsx')
else:
    grp2=None
if path.exists('Strong_Diads.xlsx'):
    grp3=pd.read_excel('Strong_Diads.xlsx')
else:
    grp3=None
df2=pd.concat([grp1, grp2, grp3], axis=0)
if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

In [4]:
df2.head()

,Unnamed: 0,filename,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,Diad1_cent_err,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,...,HB2_Sigma,C13_Cent,C13_Area,C13_Sigma,Diad2_Gauss_Cent,Diad2_Gauss_Area,Diad2_Gauss_Sigma,Diad1_Gauss_Cent,Diad1_Gauss_Area,Diad1_Gauss_Sigma
0,0,LL4-FI7-MI#1,103.146684,0.011607,0.008348,1286.099958,0.006884,197.381868,1286.099958,240.893149,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,LL7-FI28-MI#5_rep1_CRR_DiadFit,103.140286,0.027936,0.020905,1286.088345,0.018806,330.824565,1286.088345,424.918412,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,LL7-FI28-MI#5_rep2_CRR_DiadFit,103.135941,0.022480,0.016877,1286.097952,0.015249,328.845420,1286.097952,420.074411,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,LL4-FI7-MI#2_CRR_DiadFit,103.138036,0.001389,0.001389,1286.165608,0.000000,640.213299,1286.165608,749.225087,...,0.402907,1371.169059,21.527231,0.210139,NaN,NaN,NaN,NaN,NaN,NaN
0,0,LL7-FI28-MI#5_rep3_CRR_DiadFit.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## First, get the metadata for this day

In [5]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['N', 'Si', 'series','IMG','Cap','window','nodiad'],
 sort=True)
diad_meta

['LL4-FI7-MI#1.txt',
 'LL4-FI7-MI#2.txt',
 'LL7-FI28-MI#5_rep1.txt',
 'LL7-FI28-MI#5_rep2.txt',
 'LL7-FI28-MI#5_rep3.txt']

## Now get all the important information out of the metadata files

In [6]:
meta=pf.stitch_metadata_in_loop_witec(path=meta_path, 
Allfiles=diad_meta, prefix=False,
trupower=True)
meta['filename'].iloc[0]

100%|██████████| 5/5 [00:01<00:00,  3.89it/s]

Done


'LL7-FI28-MI#5_rep1.txt'

## Then get a simple file name you can stitch with the spectra

In [7]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

good job, no duplicate file names


'LL7-FI28-MI#5_rep1'

## Now get filename from spectra into same form as metadata

In [8]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

# Remove the cosmic rays, as doesnt have it in the metatdata
file_s = np.char.replace(file_s.astype(str), "_CRR_DiadFit", "")

good job, no duplicate file names


In [9]:
file_s

array(['LL4-FI7-MI#1', 'LL7-FI28-MI#5_rep1', 'LL7-FI28-MI#5_rep2',
       'LL4-FI7-MI#2', 'LL7-FI28-MI#5_rep3'], dtype='<U18')

In [10]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df2['Name_for_Secondary_Phases']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [11]:
Ne_corr=pf.calculate_Ne_corr_std_err_values(pickle_str='polyfit_data.pkl', 
    new_x=df_combo['sec since midnight'], CI=0.67)
Ne_corr.head()


1


,time,preferred_values,lower_values,upper_values
0,53123.0,0.997774,0.997769,0.997779
1,48371.0,0.997805,0.997801,0.997810
2,48716.0,0.997803,0.997798,0.997807
3,52617.0,0.997777,0.997772,0.997781
4,49078.0,0.997800,0.997795,0.997804


In [12]:

split_err=pf.propagate_errors_for_splitting(Ne_corr, df_combo)
df_combo_out=df_combo.copy()
df_combo_out.insert(1, 'Corrected_Splitting', df_combo['Splitting']*Ne_corr['preferred_values'])
df_combo_out.insert(2, 'Corr_Split+1σ', df_combo_out['Corrected_Splitting']+split_err)
df_combo_out.insert(3, 'Corr_Split-1σ',df_combo_out['Corrected_Splitting']-split_err)
df_combo_out.insert(4, 'Corr_Split_1σ_val', split_err)

nm=os.path.basename(DayFolder)
df_combo_out.to_excel(output_path+'/'+nm+'_FI_fitting.xlsx')

## If you have secondary phases, now is the time to merge those in

In [13]:
if path.exists('Carb_Peak_fits.xlsx'):
    Carb=pd.read_excel('Carb_Peak_fits.xlsx')
else:
    Carb=None
if path.exists('SO2_Peak_fits.xlsx'):
    SO2=pd.read_excel('SO2_Peak_fits.xlsx')
else:
    SO2=None
if SO2 is not None and Carb is not None:
    Sec_Phases=pd.merge(SO2, Carb, on='filename', how='outer')
elif SO2 is not None and Carb is None:
    Sec_Phases=SO2
elif SO2 is None and Carb is not None:
    Sec_Phases=Carb
else:
    Sec_Phases=None
Sec_Phases.head()

,filename,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Model_name
0,LL7-FI28-MI#5_rep1.txt,1151.241537,46.449942,31.974214,Spline
1,LL7-FI28-MI#5_rep2.txt,1151.330029,98.150975,39.076167,Spline
2,LL7-FI28-MI#5_rep3.txt,1151.300532,51.209240,39.263018,Spline


In [14]:
# Remove these to get the pure file name
if Sec_Phases is not None:
    file_sec_phase=pf.extracting_filenames_generic(
        prefix=False, str_prefix=" ",
        names=Sec_Phases['filename'].reset_index(drop=True),
       file_type='.txt')

    file_sec_phase

good job, no duplicate file names


In [15]:
file_sec_phase

array(['LL7-FI28-MI#5_rep1', 'LL7-FI28-MI#5_rep2', 'LL7-FI28-MI#5_rep3'],
      dtype=object)

In [16]:
df_combo['name_for_matching']

0          LL4-FI7-MI#1
1    LL7-FI28-MI#5_rep1
2    LL7-FI28-MI#5_rep2
3          LL4-FI7-MI#2
4    LL7-FI28-MI#5_rep3
Name: name_for_matching, dtype: object

In [17]:
df_combo['name_for_matching']=df_combo['Name_for_Secondary_Phases']

if Sec_Phases is not None:
    Sec_Phases['name_for_matching']=file_sec_phase
    df_combo_sec_phase=df_combo_out.merge(Sec_Phases, 
    on='name_for_matching', how='outer')
    
else:
    df_combo_sec_phase=df_combo_out

### Add some more useful columns

In [18]:
if 'Peak_Area_Carb' in df_combo_sec_phase.columns:
    df_combo_sec_phase['Carb_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_Carb']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
if 'Peak_Area_SO2' in df_combo_sec_phase.columns:
    df_combo_sec_phase['SO2_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_SO2']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
    
df_combo_sec_phase.head()

,Unnamed: 0,Corrected_Splitting,Corr_Split+1σ,Corr_Split-1σ,Corr_Split_1σ_val,filename_x,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,...,duration,24hr_time,sec since midnight,Spectral Center,filename,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Model_name,SO2_Diad_Ratio
0,0,102.917083,102.925427,102.908738,0.008344,LL4-FI7-MI#1,103.146684,0.011607,0.008348,1286.099958,...,"['0h', '3m', '47s']",2:45:23 PM\n,53123.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN
1,0,102.913940,102.934805,102.893076,0.020864,LL7-FI28-MI#5_rep1_CRR_DiadFit,103.140286,0.027936,0.020905,1286.088345,...,"['0h', '3m', '47s']",1:26:11 PM\n,48371.0,1325.004,LL7-FI28-MI#5_rep1.txt,1151.241537,46.449942,31.974214,Spline,0.048145
2,0,102.909318,102.926164,102.892471,0.016847,LL7-FI28-MI#5_rep2_CRR_DiadFit,103.135941,0.022480,0.016877,1286.097952,...,"['0h', '3m', '46s']",1:31:56 PM\n,48716.0,1325.004,LL7-FI28-MI#5_rep2.txt,1151.330029,98.150975,39.076167,Spline,0.099260
3,0,102.908726,102.910196,102.907256,0.001470,LL4-FI7-MI#2_CRR_DiadFit,103.138036,0.001389,0.001389,1286.165608,...,"['0h', '3m', '47s']",2:36:57 PM\n,52617.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN
4,0,NaN,NaN,NaN,NaN,LL7-FI28-MI#5_rep3_CRR_DiadFit.txt,NaN,NaN,NaN,NaN,...,"['0h', '3m', '46s']",1:37:58 PM\n,49078.0,1325.004,LL7-FI28-MI#5_rep3.txt,1151.300532,51.209240,39.263018,Spline,NaN


In [19]:
df_combo_sec_phase.to_excel(output_path+'/'+nm+'_FI_fitting_w_sec_phases.xlsx')

In [20]:
from winotify import Notification, audio

toast= Notification(app_id="VSCode",title="Notebook completed",
                    msg="Step4_Stitch_Outputs_Together is done!",
                    duration="short")
toast.set_audio(audio.Mail,loop=False)
toast.show()